# Convolutional Neural Network - Gap / Char Classification
Using TensorFlow

Also need to convert images in folder into csv data file

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2

# Creating CSV
import glob
import csv

print("OpenCV: " + cv2.__version__)
print("Numpy: " + np.__version__)
print("TensorFlow: " + tf.__version__)

OpenCV: 3.1.0-dev
Numpy: 1.12.0
TensorFlow: 1.0.0-rc1


In [2]:
def implt(img, cmp=None, t=''):
    """ Show image using plt """
    plt.imshow(img, cmap=cmp)
    plt.title(t)
    plt.show()

## Load Images and Lables in CSV

In [3]:
# 755 chars and 306 gaps images, dimension 30x60 = 1800
# Or read_csv() form pandas
images = np.genfromtxt('data/gapdet/data.csv', delimiter=',')
labels = np.genfromtxt('data/gapdet/labels.csv', delimiter=',')
labels = np.reshape(labels, (len(labels), 1))

print("CSV data files loaded.")
print("Number of images: " + str(len(images)))

div = int(0.8 * len(images))

trainData = images[0:div]
trainLabels = labels[0:div]
evalData = images[div:]
evalLabels = labels[div:]
print("Training images: %g" % div)

CSV data files loaded.
Number of images: 3413
Training images: 2730


# Create classifier

### Simoid

In [5]:
# Place holders for data and labels
x = tf.placeholder(tf.float32, [None, 1800], name = "x")
y_ = tf.placeholder(tf.float32, [None, 1], name = "y_")

# Learned variables
W = tf.Variable(tf.random_normal([1800, 1]), name = "W")
b = tf.Variable(tf.random_normal([1]), name = "b")

y = tf.matmul(x, W) + b

# Training Model
cross_entropy = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y_, logits=y, name="Cost"))
train_step = tf.train.AdamOptimizer(0.01).minimize(cross_entropy)

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

# For saving and using model
activation = tf.sigmoid(tf.matmul(x, W) + b)
tf.add_to_collection("act", activation)

for i in range(10000):
#     sess.run(train_step, feed_dict={x: images, y_: labels})
    sess.run(train_step, feed_dict={x: trainData, y_: trainLabels})
    if i % 500 == 0:
        print("%d# Cost: %g" % (i/500 + 1, sess.run(cross_entropy, feed_dict={x: trainData, y_: trainLabels})))

# Saving model
# tf.train.export_meta_graph('models/CGClassifier')
saver = tf.train.Saver()
saver.save(sess, 'models/CGClassifier')
saver.export_meta_graph('models/CGClassifier.meta')

# Evaluating model
correct_prediction = tf.equal(tf.sigmoid(y) , y_)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print("Eval accuracy: %g" % sess.run(accuracy, feed_dict={x: evalData, y_: evalLabels}))
print("Total accuracy: %g" % sess.run(accuracy, feed_dict={x: images, y_: labels}))

sess.close()

1# Cost: 1070.79
2# Cost: 27.1204
3# Cost: 5.72894
4# Cost: 1.29041
5# Cost: 1.78675
6# Cost: 8.16511e-06
7# Cost: 3.70312e-06
8# Cost: 2.2342e-06
9# Cost: 1.54497e-06
10# Cost: 1.16644e-06
11# Cost: 9.33319e-07
12# Cost: 7.56638e-07
13# Cost: 6.13491e-07
14# Cost: 4.95996e-07
15# Cost: 3.99948e-07
16# Cost: 3.21905e-07
17# Cost: 2.58889e-07
18# Cost: 2.08237e-07
19# Cost: 1.67574e-07
20# Cost: 1.34827e-07
Eval accuracy: 0.650073
Total accuracy: 0.823616


### Sofmax

In [9]:
def sofmaxLabels(lab):
    """ Transform data for sofmax classification """
    newLab = np.zeros((len(lab), 2), dtype=np.int8)
    for i in range(len(lab)):
        newLab[i][int(lab[i][0])] = 1
    return newLab

llabels = sofmaxLabels(labels)
trainLabels = llabels[0:div]
evalLabels = llabels[div:]

In [11]:
sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32, [None, 1800])
y_ = tf.placeholder(tf.float32, [None, 2])

W = tf.Variable(tf.random_normal([1800, 2]))
b = tf.Variable(tf.random_normal([2]))

y = tf.matmul(x,W) + b

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

train_step = tf.train.AdamOptimizer(0.01).minimize(cross_entropy)


sess.run(tf.global_variables_initializer())

for i in range(10000):
    sess.run(train_step, feed_dict={x: trainData, y_: trainLabels})
    if i % 500 == 0:
        print("%d# Cost: %g" % (i/500 + 1, sess.run(cross_entropy, feed_dict={x: trainData, y_: trainLabels})))

    
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Eval accuracy: %g" % sess.run(accuracy, feed_dict={x: evalData, y_: evalLabels}))
print("Total accuracy: %g" % sess.run(accuracy, feed_dict={x: images, y_: llabels}))

sess.close()

1# Cost: 1549.94
2# Cost: 22.6148
3# Cost: 3.06887
4# Cost: 0.364619
5# Cost: 2.29443e-06
6# Cost: 1.0432e-06
7# Cost: 6.60965e-07
8# Cost: 4.62552e-07
9# Cost: 3.38586e-07
10# Cost: 2.55141e-07
11# Cost: 1.96192e-07
12# Cost: 1.5222e-07
13# Cost: 1.18903e-07
14# Cost: 9.35332e-08
15# Cost: 7.38834e-08
16# Cost: 5.79889e-08
17# Cost: 4.5806e-08
18# Cost: 3.62868e-08
19# Cost: 2.89071e-08
20# Cost: 2.25319e-08
Eval accuracy: 0.682284
Total accuracy: 0.93642


### Convulation Neural Network

In [12]:
sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32, [None, 1800])

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

x_image = tf.reshape(x, [-1, 30, 60, 1])

# 1. Layer
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# 2. Layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# Connected layer
W_fc1 = weight_variable([8 * 15 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 8*15*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Output layer
W_fc2 = weight_variable([1024, 2])
b_fc2 = bias_variable([2])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
y_ = tf.placeholder(tf.float32, [None, 2])

# Train
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv + 1e-10))
train_step = tf.train.AdamOptimizer(1e-2).minimize(cross_entropy)   # 1e-4 - default

correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess.run(tf.global_variables_initializer())

for i in range(200):
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x: trainData, y_: trainLabels, keep_prob: 1.0})
        print("step %d, training accuracy %g" % (i, train_accuracy))
    train_step.run(feed_dict={x: trainData, y_: trainLabels, keep_prob: 0.5})

print("Test accuracy %g" % accuracy.eval(feed_dict={x: evalData, y_: evalLabels, keep_prob: 1.0}))
print("Total accuracy: %g" % accuracy.eval(feed_dict={x: images, y_: llabels, keep_prob: 1.0}))

sess.close()

step 0, training accuracy 0.322711


KeyboardInterrupt: 